In [42]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import *
from tensorflow.keras.applications import ResNet50

In [43]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
!cp -r /content/drive/MyDrive/archive.tar /content/

In [45]:
!tar -xf /content/archive.tar -C /content/

In [46]:
train_dir = "/content/data/fruits-360_100x100/fruits-360/Training"
test_dir = "/content/data/fruits-360_100x100/fruits-360/Test"

In [49]:
train_tfms = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.Lambda(
        tf.keras.applications.resnet50.preprocess_input
    )
])


val_tfms = tf.keras.Sequential([
    tf.keras.layers.Lambda(
        tf.keras.applications.resnet50.preprocess_input
    )
])


In [50]:
!cp -r /content/drive/MyDrive/Fruits-360_/model/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5 /content/

In [51]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

In [52]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)

num_classes = len(train_ds.class_names)
print(f"Number of classes: {num_classes}")

Found 122118 files belonging to 232 classes.
Number of classes: 232


In [53]:
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 232)            │       119,016 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,764,008 (94.47 MB)

 Trainable params: 1,172,200 (4.47 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

In [54]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32
)

train_ds = train_ds.map(
    lambda x, y: (train_tfms(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

val_ds = test_ds.map(
    lambda x, y: (val_tfms(x, training=False), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

Found 40715 files belonging to 232 classes.
Epoch 1/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1037s 268ms/step - accuracy: 0.7593 - loss: 1.1033 - val_accuracy: 0.9808 - val_loss: 0.0585
Epoch 2/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1022s 268ms/step - accuracy: 0.9796 - loss: 0.0642 - val_accuracy: 0.9885 - val_loss: 0.0434
Epoch 3/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1047s 274ms/step - accuracy: 0.9834 - loss: 0.0478 - val_accuracy: 0.9899 - val_loss: 0.0361
Epoch 4/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1024s 268ms/step - accuracy: 0.9877 - loss: 0.0408 - val_accuracy: 0.9924 - val_loss: 0.0315
Epoch 5/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1054s 276ms/step - accuracy: 0.9894 - loss: 0.0347 - val_accuracy: 0.9902 - val_loss: 0.0386


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc}")

In [59]:
model.save('/content/classifier.keras')

In [60]:
!cp -r /content/classifier.keras /content/drive/MyDrive/Fruits-360_